In [1]:
!pip install yfinance

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

def download_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    returns = data.pct_change().dropna()
    return returns

def calculate_portfolio_stats(returns):
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    return mean_returns, cov_matrix

#tickers = ['PRIO3.SA','JHSF3.SA','ODPV3.SA','ITSA4.SA', 'EGIE3.SA' ,'LREN3.SA' ,'FLRY3.SA' ,'TRIS3.SA','SBFG3.SA',           'BBAS3.SA' ,'VALE3.SA' ,'ALOS3.SA','KLBN4.SA','CSAN3.SA','IVVB11.SA']

#returns = download_data(tickers, "2020-01-01", "2025-05-31")

tickers = ['AMZN', 'GOOG', 'MSFT', 'NFLX']
returns = download_data(tickers, "2020-01-01", "2024-12-31")

# mean_returns -> média  Pandas.DataSeries
# cov_matrix -> matriz de covariancia Dataframe
mean_returns, cov_matrix = calculate_portfolio_stats(returns)
print('Médias:' )
print( mean_returns)
print('matriz de covariancia: ')
print(cov_matrix)


/tmp/ipython-input-2-758755222.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed

Médias:
Ticker
AMZN    0.000931
GOOG    0.001036
MSFT    0.000995
NFLX    0.001226
dtype: float64
matriz de covariancia: 
Ticker      AMZN      GOOG      MSFT      NFLX
Ticker                                        
AMZN    0.000514  0.000301  0.000295  0.000350
GOOG    0.000301  0.000416  0.000292  0.000263
MSFT    0.000295  0.000292  0.000369  0.000267
NFLX    0.000350  0.000263  0.000267  0.000823


In [3]:
# cria uma llista de pesos de soma 1

def generate_random_weights(num_assets):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    return weights
weights = generate_random_weights(len(tickers))
weights

array([0.36022797, 0.1154487 , 0.33758799, 0.18673535])

image.png


In [4]:
def calculate_portfolio_performance(weights, mean_returns, cov_matrix):
    #calcula o retorno esperado da carteira, como uma soma ponderada dos retornos médios dos ativos.
    portfolio_return = np.dot(weights, mean_returns)

    # np.dot(cov_matrix, weights) -> vetor resultante da matriz de covariância multiplicada pelos pesos
    # — ele representa a covariância ponderada de cada ativo com a carteira.

    # np.dot(weights.T, A) -> resultante da transporta dos pesos pela covariância ponderada
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    sharpe_ratio = portfolio_return / portfolio_stddev
    return portfolio_return, portfolio_stddev, sharpe_ratio

#exemplos
weights = generate_random_weights(len(tickers))
portfolio_return, portfolio_stddev, sharpe_ratio = calculate_portfolio_performance(weights, mean_returns, cov_matrix)
portfolio_return, portfolio_stddev, sharpe_ratio

(np.float64(0.0010036748050631689),
 np.float64(0.018238221872637416),
 np.float64(0.05503139571785614))

In [5]:
# Você está criando uma matriz results com 3 linhas e 10.000 colunas, inicialmente cheia de zeros.
# Essa matriz vai guardar os resultados de cada portfólio simulado, normalmente:

num_portfolios = 10000
results = np.zeros((3, num_portfolios))
lista_pesos=[]

for i in range(num_portfolios):
    weights = generate_random_weights(len(tickers))
    portfolio_return, portfolio_stddev, sharpe_ratio = calculate_portfolio_performance(weights, mean_returns, cov_matrix)

    results[0,i] = portfolio_return
    results[1,i] = portfolio_stddev
    results[2,i] = sharpe_ratio
    lista_pesos.append(weights)

In [6]:
# plota a fronteira eficiente

plt.figure(figsize=(10, 6))
plt.scatter(results[1,:], results[0,:], c=results[2,:], cmap='YlGnBu', marker='o')
plt.title('Fronteira Eficiente')
plt.xlabel('Risco (Desvio Padrão)')
plt.ylabel('Retorno Esperado')
plt.colorbar(label='Sharpe Ratio')
plt.show()

In [ ]:
max_sharpe_idx = np.argmax(results[2,:]) #busca o indice que corresponde ao maior Sharp
sdp, rp = results[1,max_sharpe_idx], results[0,max_sharpe_idx]
print(f'Retorno do portfólio ótimo: {round(rp,4)}')
print(f'Risco do portfólio ótimo: {round(sdp,4)}')

# acho que esta errado
# optimal_weights = generate_random_weights(len(tickers))
# pesos_np = np.round(optimal_weights, 4)
# print(f'Pesos do portfólio ótimo: {pesos_np}')

df = pd.DataFrame({'Ticket': tickers, 'Peso': lista_pesos[max_sharpe_idx]})
df['Perct_Peso'] = df['Peso'] * 100
df[['Ticket', 'Perct_Peso']]


Retorno do portfólio ótimo: 0.0022
Risco do portfólio ótimo: 0.015


Ticket  Perct_Peso
0   AMZN   25.646605
1   GOOG   12.141261
2   MSFT   42.338320
3   NFLX   19.873814

#Diversos

In [ ]:
df = pd.DataFrame(results).T
df.columns = ['Retorno', 'Risco', 'Sharpe']
df.sort_values(by='Risco', ascending=True, inplace=True)
df.head(5000)


Retorno     Risco    Sharpe
526   0.000402  0.012435  0.032340
9842  0.000379  0.012840  0.029479
5548  0.000346  0.012851  0.026929
5589  0.000378  0.013232  0.028590
3473  0.000432  0.013378  0.032266
...        ...       ...       ...
4170  0.000153  0.016916  0.009035
7930  0.000364  0.016916  0.021512
3099  0.000333  0.016916  0.019677
3986  0.000245  0.016917  0.014503
793   0.000318  0.016917  0.018819

[5000 rows x 3 columns]

In [ ]:
df = pd.DataFrame({'Ticket': tickers, 'Peso': lista_pesos[526]})
df

Ticket      Peso
0    PRIO3.SA  0.099110
1    JHSF3.SA  0.062582
2    ODPV3.SA  0.000990
3    ITSA4.SA  0.169227
4    EGIE3.SA  0.042803
5    LREN3.SA  0.071623
6    FLRY3.SA  0.195815
7    TRIS3.SA  0.018943
8    SBFG3.SA  0.041649
9    BBAS3.SA  0.004128
10   VALE3.SA  0.090036
11   ALOS3.SA  0.026432
12   KLBN4.SA  0.009258
13   CSAN3.SA  0.004744
14  IVVB11.SA  0.162660